### imports

In [10]:
import pandas as pd
import geopandas as gpd
import altair as alt
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
import scikit_posthocs as sp

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import plotly.graph_objects as go

alt.data_transformers.enable("vegafusion")
alt.data_transformers.disable_max_rows()

df = pd.read_csv("amostra.csv", index_col=0)

C:\Users\muril\AppData\Local\Temp\ipykernel_14056\241172229.py:16: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("amostra.csv", index_col=0)


In [ ]:
def set_global_chart_config():
    return {
        'config': {
            'title': {'fontSize': 18, 'font': 'Arial', 'anchor': 'start', 'color': 'black'},
            'axis': {
                'labelFontSize': 12,
                'titleFontSize': 14,
                'labelFont': 'Arial',
                'titleFont': 'Arial'
            },
            'legend': {
                'titleFontSize': 14,
                'labelFontSize': 12,
                'labelFont': 'Arial',
                'titleFont': 'Arial'
            }
        }
    }
    
# Aplicar a configuração global
alt.themes.register('custom_theme', lambda: set_global_chart_config())
alt.themes.enable('custom_theme')

ThemeRegistry.enable('custom_theme')

### Very high cardinality variables

In [43]:
def plot_top_categories(df, column, top_n=12):
    
    color_palette = ["#0b132b", "#152139", "#202e47", 
                    "#2a3c56", "#354964", "#3a506b",
                    "#467787", "#4c8a95",  "#57b1b1",
                     "#5dc5bf", "#63d8cd", "#69ecdb", "#6fffe9"]
    
    vis = df[column].value_counts().reset_index().head(top_n)
    
    color_scale = alt.Scale(
        domain=vis[column].unique(),
        range=color_palette    
    )

    highlight = alt.selection_point(on='mouseover', fields=[column], empty='none')
    chart = alt.Chart(vis).mark_bar().encode(
        x=alt.X(f"{column}:O", title=column.capitalize(), sort='-y'),
        y=alt.Y("count:Q", title="Contagem das classes"),
        
        tooltip=[
            alt.Tooltip(f"{column}:O"),
            alt.Tooltip("count:Q", title="Contagem da classe -")
        ],
        color=alt.condition(
            highlight,
            alt.Color(f"{column}:O", scale=color_scale, title=f'{column}'),
            alt.value('lightgray')
        ),
                
    ).properties(
        width=800,
        height=300,
    ).add_params(
        highlight
    ).configure(
    background="rgba(0, 0, 0, 0)"
    ).configure_view(
        fill='white'
    ).configure_axis(
        grid=False,
    ).configure_axis(
        labelAngle=-45
    )
    
    return chart
vis1 = plot_top_categories(df, 'occupation')
vis1.display()

alt.Chart(...)

In [59]:
import altair as alt
import pandas as pd

def cumulative_percentage_lineplot(df, column):
    value_counts = df[column].value_counts(ascending=True)
    total = value_counts.sum()
    individual_percentage = (value_counts / total) * 100
    cumulative_percentage = (value_counts / total).cumsum() * 100
    
    cumulative_df = pd.DataFrame({
        column: value_counts.index,
        'individual_percentage': individual_percentage.values,
        'cumulative_percentage': cumulative_percentage.values,
        'index': range(len(value_counts))  # Adiciona um índice numérico para usar como eixo X
    }).reset_index(drop=True)
    
    # Cria o gráfico de linha com eixo X numérico
    line = alt.Chart(cumulative_df).mark_line().encode(
        x=alt.X('index:Q', title=column.capitalize(), axis=alt.Axis(labels=False, grid=False)),
        y=alt.Y('cumulative_percentage:Q', title='Cumulative Percentage (%)'),
        tooltip=[alt.Tooltip(f"{column}:O", title=column.capitalize()), 
                 alt.Tooltip("individual_percentage:Q", title="Individual Percentage (%)", format=".2f"),
                 alt.Tooltip("cumulative_percentage:Q", title="Cumulative Percentage (%)", format='.2f')]
    )
    
    # Adiciona pontos ao gráfico de linha
    points = alt.Chart(cumulative_df).mark_point(size=50).encode(
        x=alt.X('index:Q'),
        y=alt.Y('cumulative_percentage:Q'),
        tooltip=[alt.Tooltip(f"{column}:O", title=column.capitalize()),
                 alt.Tooltip("individual_percentage:Q", title="Individual Percentage (%)", format=".2f"), 
                 alt.Tooltip("cumulative_percentage:Q", title="Cumulative Percentage (%)", format='.2f')]
    )
    
    # Combina os gráficos de linha e pontos, com interatividade completa
    chart = (line + points).properties(
        title=f'Cumulative Percentage Distribution of {column.capitalize()}',
        width=800,
        height=400
    ).configure(
        background="rgba(0, 0, 0, 0)"
    ).configure_view(
        fill='white'
    ).configure_axis(
        labelAngle=-45,
    ).interactive()
    
    return chart

# Exemplo de uso
vis2 = cumulative_percentage_lineplot(df, 'occupation')
vis2.display()


alt.LayerChart(...)

### Geospatial analysis

In [11]:
state_boundaries = gpd.read_file(r"cb_2018_us_state_500k\cb_2018_us_state_500k.shp", columns=['STUSPS', 'NAME', 'geometry'])
state_boundaries.rename(columns={'STUSPS': 'state',
                           'Name': 'state_name'}, inplace=True)

df = state_boundaries.merge(df, on='state', how='inner')
df.columns

Index(['NAME', 'state', 'geometry', 'zip_code', 'latitude', 'longitude',
       'cmte_id', 'amndt_ind', 'rpt_tp', 'transaction_pgi', 'transaction_tp',
       'entity_tp', 'city', 'employer', 'occupation', 'transaction_dt',
       'transaction_amt', 'file_num'],
      dtype='object')

In [12]:
agg_state = df.groupby('state', as_index=False).agg({
    'transaction_amt': 'sum',
    'geometry': 'last'
})
agg_state

,state,transaction_amt,geometry
0,AK,2235.282967,"MULTIPOLYGON (((179.48246 51.98283, 179.48656 ..."
1,AL,2410.423817,"MULTIPOLYGON (((-88.05338 30.50699, -88.05109 ..."
2,AR,2481.961170,"POLYGON ((-94.61783 36.49941, -94.61765 36.499..."
3,AS,27.525661,"MULTIPOLYGON (((-168.14582 -14.54791, -168.145..."
4,AZ,5008.656380,"POLYGON ((-114.81629 32.50804, -114.81432 32.5..."
5,CA,112979.859670,"MULTIPOLYGON (((-118.60442 33.47855, -118.5987..."
6,CO,10754.745255,"POLYGON ((-109.06025 38.59933, -109.05954 38.7..."
7,CT,9723.902098,"MULTIPOLYGON (((-72.76143 41.24233, -72.75973 ..."
8,DC,1931.545483,"POLYGON ((-77.11976 38.93434, -77.11253 38.940..."
9,DE,3941.113092,"MULTIPOLYGON (((-75.56555 39.51485, -75.56174 ..."
